## Regex & Filtrage ##

In [ ]:
import re

# Liste des produits de l'exercice
products = [
    "panatalon noir", "balai essuie glaces avant", "fromage fondu kiri", 
    "lentilles 265g", "croutons à l'ail tipiak", "mozarella bille 150g", 
    "sac a bandouillere en nylon", "mais doux saint eloi", 
    "croustibat findus", "pipe rigate carrefour"
]

def clean_and_tokenize(text):
    # Suppression des  caractères spéciaux
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    # Suppression des chiffres par exemple 150g
    text = re.sub(r'\d+', '', text)
    # Tokenisation + suppression des mots < 3 lettres
    tokens = [w for w in text.lower().split() if len(w) >= 3]
    return tokens

# Création de la liste filtrée
filtered_products = [" ".join(clean_and_tokenize(p)) for p in products]

#display de la liste filtrée
print("--- PRODUITS FILTRÉS ---")
for i, p in enumerate(filtered_products):
    print(f"{i}: {p}")

--- PRODUITS FILTRÉS ---
0: panatalon noir
1: balai essuie glaces avant
2: fromage fondu kiri
3: lentilles
4: croutons ail tipiak
5: mozarella bille
6: sac bandouillere nylon
7: mais doux saint eloi
8: croustibat findus
9: pipe rigate carrefour


## Matrice TF - IDF ##

In [ ]:
import math

# Extraction du vocabulaire "unique"
vocab = sorted(list(set([word for p in filtered_products for word in p.split()])))
N = len(filtered_products)

# On défini la fonction pour calculer les scores TF-IDF
def get_tfidf(doc_str):
    words = doc_str.split()
    vector = []
    for term in vocab:
        # Term Frequency : nb d'occurrences du mot / total de mots dans ce produit
        # on rajoute une condition if pour éciter l'erreur de division par 0
        term_frequency = words.count(term) / len(words) if len(words) > 0 else 0
        
        # Inverse Document Frequency : log(Total produits / nb produits contenant le mot)
        # On compte dans combien de produits du corpus le mot apparaît
        df = sum(1 for product in filtered_products if term in product.split())
        idf = math.log(N / df) if df > 0 else 0
        
        # Le score final est la multiplication des deux
        vector.append(term_frequency * idf)
    return vector

# On génère la matrice complete 
tfidf_matrix = [get_tfidf(p) for p in filtered_products]

# On initie le formatage de l'affichage de la matrice en montrant explicitement la taille
print(f"MATRICE TF-IDF ({N} produits x {len(vocab)} mots)\n")

# On crée l'en tête avec les mots de la liste de vocabulaire unique
# On utilise {:<25} pour aligner le texte à gauche sur 25 caractères
header = "{:<25}".format("PRODUIT / MOTS") 
for word in vocab:
    # On utilise {:>12} pour aligner le texte à droite sur 12 caractères
    header += "{:>12}".format(word)
print(header)
print("-" * len(header))

# Affichage des lignes (Produit + scores)
for i, vector in enumerate(tfidf_matrix):
    # On affiche le nom du produit
    row = "{:<25}".format(filtered_products[i][:24])
    for score in vector:
        # On utilise {:>12.2f} pour aligner le score à droite sur 12 caractères avec 2 décimales
        row += "{:>12.2f}".format(score)
    print(row)

MATRICE TF-IDF (10 produits x 27 mots)

PRODUIT / MOTS                    ail       avant       balaibandouillere       bille   carrefour  croustibat    croutons        doux        eloi      essuie      findus       fondu     fromage      glaces        kiri   lentilles        mais   mozarella        noir       nylon   panatalon        pipe      rigate         sac       saint      tipiak
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
panatalon noir                   0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00        0.00       

## Similarity Cosinus ##

$$\text{similarity} = \frac{A \cdot B}{\|A\| \|B\|}$$

In [ ]:
import math

# 1. Fonctions mathématiques pour le calcul du Cosinus
def dot_product(v1, v2):
    # Calcule la somme des produits des éléments correspondants (A * B)
    #zip permet de parcourir les deux vecteurs en parallèle
    return sum(a * b for a, b in zip(v1, v2))

def norme(v):
    # Calcule la norme du vecteur. la norme est la longueur du vecteur dans l'espace
    return math.sqrt(sum(a**2 for a in v))

def cosine_sim(v1, v2):
    # (A . B) / (||A|| * ||B||)
    m1, m2 = norme(v1), norme(v2)
    if m1 == 0 or m2 == 0: 
        return 0.0
    return dot_product(v1, v2) / (m1 * m2)

# On compare les produits entre eux
print("--- RÉSULTATS DE SIMILARITÉ (BASÉS SUR LE TF-IDF) ---\n")

for i in range(len(filtered_products)):
    similarities = []
    
    # On compare le produit i avec TOUS les autres produits j
    for j in range(len(filtered_products)):
        if i == j: 
            continue # On ignore la comparaison avec soi-même
        
        # Calcul du score entre le vecteur i et le vecteur j de la matrice
        score = cosine_sim(tfidf_matrix[i], tfidf_matrix[j])
        similarities.append((score, filtered_products[j]))
    
    # On cherche le score maximum trouvé pour le produit
    # Si tous les scores sont à 0.00 alors il prendra le premier résultat par défaut
    best_score, best_name = max(similarities, key=lambda x: x[0])
    
    # On affiche le score final
    print(f"Produit analysé : '{filtered_products[i]}'")
    # Si le score est supérieur à 0 on affiche le produit le plus proche avec son score
    if best_score > 0:
        print(f" -> Produit le plus proche : '{best_name}' (Score de similarité : {best_score:.2f})")
    else:
        # sinon on indique q'aucun produit similaire n'a été trouvé et on affiche un score égal à 0
        print(f" -> Aucun produit similaire trouvé (Score : 0.00)")
    # juste une ligne de tirés pour séparer les analyses produits
    print("-" * 50)

--- RÉSULTATS DE SIMILARITÉ (BASÉS SUR LE TF-IDF) ---

Produit analysé : 'panatalon noir'
 -> Aucun produit similaire trouvé (Score : 0.00)
--------------------------------------------------
Produit analysé : 'balai essuie glaces avant'
 -> Aucun produit similaire trouvé (Score : 0.00)
--------------------------------------------------
Produit analysé : 'fromage fondu kiri'
 -> Aucun produit similaire trouvé (Score : 0.00)
--------------------------------------------------
Produit analysé : 'lentilles'
 -> Aucun produit similaire trouvé (Score : 0.00)
--------------------------------------------------
Produit analysé : 'croutons ail tipiak'
 -> Aucun produit similaire trouvé (Score : 0.00)
--------------------------------------------------
Produit analysé : 'mozarella bille'
 -> Aucun produit similaire trouvé (Score : 0.00)
--------------------------------------------------
Produit analysé : 'sac bandouillere nylon'
 -> Aucun produit similaire trouvé (Score : 0.00)
--------------------

In [ ]:
import pandas as pd
import numpy as np
import os

# on defini la fonction de la distance de Levenshtein
def levenshtein(s, t):
    #on convertit les inputs en string
    s, t = str(s), str(t)
    # on récupère les longueurs des deux inputs
    n, m = len(s), len(t)
    # on initialise la matrice des distances à origine 0
    d = np.zeros((n + 1, m + 1), dtype=int)
    # on remplit la première ligne
    for i in range(n + 1): d[i, 0] = i
    # on remplit la première colonne
    for j in range(m + 1): d[0, j] = j
    # on remplit le reste de la matrice
    for j in range(1, m + 1):
        # on parcout chaque caractère du premier string
        for i in range(1, n + 1):
            # si les caractères sont égaux alors le "coût" est de 0 sinon 1
            if s[i-1] == t[j-1]:
                cost = 0
            else:
                cost = 1
            # on calcule la valeur minimale parmi les 3 opérations possibles du levenshtein
            d[i, j] = min(d[i-1, j] + 1,      # suppression
                          d[i, j-1] + 1,      # insertion
                          d[i-1, j-1] + cost) # substitution
    return d[n, m]

# on récupère les chemins des fichiers car ils ne sont pas à la meêm racine que le notebook
dossier_parent = os.getcwd()
chemin_csv = os.path.join(dossier_parent, 'fichiers_csv', 'levenshtein_pairs.csv')
chemin_sortie = os.path.join(dossier_parent, 'fichiers_csv', 'levenshtein_results.csv')

# --- APPLICATION AU CSV ---
df_pairs = pd.read_csv(chemin_csv)

# On applique la fonction sur chaque ligne. axis=1 sert à appliquer la fonction ligne par ligne
df_pairs['distance'] = df_pairs.apply(lambda row: levenshtein(row['string1'], row['string2']), axis=1)

# On sauvegarde le résultat demandé
df_pairs.to_csv(chemin_sortie, index=False)
print("Fichier 'levenshtein_results.csv' engistré")

pd.read_csv(chemin_sortie)

Fichier 'levenshtein_results.csv' engistré


,string1,string2,distance
0,kitten,sitting,3
1,flaw,lawn,2
2,intention,execution,5
3,gumbo,gambol,2
4,rosettacode,raisethysword,8
5,Saturday,Sunday,3
6,book,back,2
7,algorithm,logarithm,3
8,distance,instance,2
9,case,caste,1
